In [446]:
!pip install psycopg2-binary
!apt install postgresql postgresql-contrib &>log
!service postgresql start
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER "
!sudo -u postgres psql -c "CREATE DATABASE test6"

 * Starting PostgreSQL 10 database server
   ...done.
ERROR:  role "root" already exists
ERROR:  database "test6" already exists


In [447]:
import psycopg2
import pandas as pd
import numpy as np
from array import array
import random
adm_role = "ADMIN"
user_role = "USER"

Так как в задании не было всех требований к ведению ролевой модели, то считаем, что роли ведутся организационным способом вручную. То есть считаем, что все роли добавляет АИБ вручную и что он  следит за их корректностью.

Аналогично для списка товаров (кофе)

PS если останется время, то смогу добавить проверки на корректность.

In [583]:
class CoffeeHouse:
  #функции для запуска скриптов
  def __execute_sql_script(self, s):
    cursor = self.__conn.cursor()
    try:
      cursor.execute(s)
    finally:
      self.__conn.commit()
      cursor.close()
  def __execute_sql_script_and_return_result(self, s):
    cursor = self.__conn.cursor()
    try:
      cursor.execute(s)
      data=cursor.fetchall()
    finally:
      self.__conn.commit()
      cursor.close()
    return data
  def __drop_table(self, table_name):
    cursor = self.__conn.cursor()
    q = "drop table if exists " + table_name + ";"
    try:
      cursor.execute(q)
    finally:
      cursor.close()
      self.__conn.commit()
  #функции для добавления записей в таблицы + функция вывода всех данных из таблицы
  def __create_insert_query(self, table_name, values):
    sql_query="insert into " + table_name + " values("
    for v in values: 
      type_v = str(type(v))
      str_v = str(v)
      #print(type_v.find("str"))
      #print(str_v)
      if type_v.find("str") != -1:
        sql_query += '\''
      sql_query += str_v
      if type_v.find("str") != -1:
        sql_query += '\''
      sql_query += ", "
    sql_query = sql_query[:-2]
    sql_query += ");";
    return sql_query
  def __insert_into_table(self, table_name, values):
      q = self.__create_insert_query(table_name, values)
      #print(q)
      cursor = self.__conn.cursor()
      cursor.execute(q)
      cursor.close()
      self.__conn.commit()
  def __print_all_data_from_table(self, table_name):
    cursor = self.__conn.cursor()
    q = "select * from " + table_name + ";"
    cursor.execute(q)
    data=cursor.fetchall()
    print(data)
    cursor.close()
    self.__conn.commit()

  #функции для заполнения таблицы с ролями
  def __get_max_value(self, table_name, column):
    q = "select max(" + column + ") from " + table_name + ";"
    #print(q)
    tmp_res = self.__execute_sql_script_and_return_result(q)
    #print(tmp_res[0][0])
    return tmp_res[0][0]
    

  def __get_next_id(self, table_name, column="id"):
    id = self.__get_max_value(table_name, column)
    if not id:
      id = 1
    else:
      id += 1
    return id

  def __add_new_person(self, name, login, psw, role):
    id = self.__get_next_id("staff", "id")
    q = "insert into staff (id, login, psw, role, name, t_start, t_end) values(" + str(id) + ", '" + login + "', '" + psw + "', '" + role+ "', '" + name + "', CURRENT_TIMESTAMP, NULL);"
    self.__execute_sql_script(q)
    #print(q)  

  #Функция для проверки роли пользователя
  def __get_role(self, id):
    q = "select role from staff where id = " + str(id) + ";"
    #print(q)
    tmp_res = self.__execute_sql_script_and_return_result(q)
    if len(tmp_res) == 0:
      return ""
    return tmp_res[0][0] 

  def __start_shift(self, staff_id):
    #проверить есть ли у пользователя права па запуск смены
    if self.__get_role(staff_id) != user_role:
      return (False, "У вас нет прав на откртытие/закрытие новой смены. Для обновления прав доступа обратитесь к АИБ системы")
    #проверить есть ли для этого id уже запущена смена
    tmp_res = self.__execute_sql_script_and_return_result("select max(t_start) from shift where shift.staff_id = " + str(staff_id) + " and t_end is NULL;")
    if not tmp_res[0][0]:
      id = self.__get_next_id("shift", "id")
      q = "insert into shift (id, staff_id, t_start, t_end) values(" + str(id) + ", " + str(staff_id) + ", CURRENT_TIMESTAMP, NULL);"
      self.__execute_sql_script(q)
      return (True, q)
    else:
      return (False, "Ваша смена уже запущена") 

  def __close_shift(self, staff_id):
    #проверить есть ли у пользователя права па запуск смены
    if self.__get_role(staff_id) != user_role:
      return (False, "У вас нет прав на откртытие/закрытие новой смены. Для обновления прав доступа обратитесь к АИБ системы")
    #проверяем, что у данного пользователя есть запущенная смена
    tmp_res = self.__execute_sql_script_and_return_result("select id, max(t_start) from shift where shift.staff_id = " + str(staff_id) + " and t_end is NULL GROUP BY id;")
    #print(tmp_res)
    if len(tmp_res) != 0:
      q = "update shift set t_end = CURRENT_TIMESTAMP where staff_id = " + str(staff_id) + " and t_end is NULL;"
      self.__execute_sql_script(q)
      return (True, q)
    else:
      return (False, "У вас нет запущенных смен")

  def __add_new_type_of_coffee(self, name, price, desc = ""):
    id = self.__get_next_id("types_of_coffee", "id")
    q = "insert into types_of_coffee (id, name, price, description, t_start, t_end) values (" + str(id) + ", '" + name + "', " + str(price) + ", '" + desc + "', CURRENT_TIMESTAMP, NULL);"
    #print(q)
    self.__execute_sql_script(q)
  
  def __union_orders(self, arr):
    c = dict()
    for i in arr:
      #print(i)
      if (len(i) == 1):
        cnt = 0
      else:
        cnt = i[1]
      if i[0] in c.keys():
        c[i[0]] += cnt
      else: 
        c[i[0]] = cnt
    res = [[i, c[i]] for i in c.keys()]
    return res
    
  def __make_order(self, staff_id, order=[]):
    order = self.__union_orders(order)
    #проверить есть ли у пользователя права на введение заказа
    if self.__get_role(staff_id) != user_role:
      return (False, "У вас нет прав на создлание заказа. Для обновления прав доступа обратитесь к АИБ системы")
    #проверяем, что у пользователя есть запущенная смена
    tmp_res = self.__execute_sql_script_and_return_result("select id from shift where shift.staff_id = " + str(staff_id) + " and t_end is NULL;")
    #print(tmp_res)
    non_existent_coffee = []
    existent_coffee = []
    if len(tmp_res) != 0:
      for o in order:
        #print(o)
        #проверяем, что такой кофе существует 
        coffee = self.__execute_sql_script_and_return_result("select id from types_of_coffee where types_of_coffee.id = " + str(o[0]) + " and t_end is NULL;")
      # print(coffee)
        if len(coffee) != 0:
          existent_coffee.append(o)
        else:
          non_existent_coffee.append(o)
      if len(non_existent_coffee) != 0:
        res = "Следующих видов кофе нет в меню (id): "
        for c in non_existent_coffee:
          res += str(c[0])
          res += ", "
        res = res[:-2]
        res += ". Обновите меню или заказ"
        return (False, res)
      else:
        shift_id = self.__execute_sql_script_and_return_result("select id from shift where shift.staff_id = " + str(staff_id) + " and t_end is NULL;")[0][0]
        #print(shift_id)
        id = self.__get_next_id("sales", "id")
        #print(id)
        # ПОДУМАТЬ
        ex_q = []
        for c in existent_coffee:
          q = "insert into sales (id, item_id, id_shift, cnt, time) values (" + str(id) + ", " + str(c[0]) + ", " + str(shift_id) + ", " + str(c[1]) + ", CURRENT_TIMESTAMP);"
          #print(q)
          self.__execute_sql_script(q)
          ex_q.append(q)
          #print(c)

        #q = insert()
        return (True, ex_q)
    else:
      return (False, "У вас нет запущенных смен. Для формирования заказа запустите смену")

  def __make_str_for_request(self, arr):
    if len(arr) > 0:
      str_arr = ""
      for id in arr:
          str_arr += str(id)
          str_arr += ", " 
      return str_arr[:-2]
    else: 
      return ""

  def __get_sales(self, staff_id, cashiers_id, shifts_id, items_id):
    if len(cashiers_id) == 0:
      cashiers_id = [c[0] for c in self.__execute_sql_script_and_return_result("select distinct id from staff")]
    if len(shifts_id) == 0:
      shifts_id = [c[0] for c in self.__execute_sql_script_and_return_result("select distinct id from shift")]
   
    if len(items_id) == 0:
      items_id = [c[0] for c in self.__execute_sql_script_and_return_result("select distinct id from types_of_coffee")]
    if self.__get_role(staff_id) != adm_role:
      return (False, "У вас нет прав на создлание заказа. Для обновления прав доступа обратитесь к АИБ системы")
    q = "select id from shift where staff_id in (" + self.__make_str_for_request(cashiers_id) + ");"
    cashiers_shifts_id = self.__execute_sql_script_and_return_result(q)
    final_shifts_id = set(shifts_id)
    for c in cashiers_shifts_id:
      final_shifts_id.add(c[0])
    #print(final_shifts_id)
    str_items_id = self.__make_str_for_request(items_id)
    str_shifts_id = self.__make_str_for_request(final_shifts_id)
    str_cashiers_id = self.__make_str_for_request(cashiers_id)
    #print(cashiers_id)
    #print(final_shifts_id)
    #print(items_id)

    q = """select a.id as id,
                  a.item_id as item_id, 
                  types_of_coffee.name as name, 
                  a.id_shift as id_shift,
                  a.cnt as cnt, 
                  a.time as time from (select distinct * from sales where id_shift in ("""
    q += str_shifts_id + ") and item_id in (" + str_items_id 
    q += """)) as a INNER JOIN types_of_coffee ON types_of_coffee.id = a.item_id;"""
    q2 = """select b.id as id,
                  b.item_id as item_id, 
                  b.name as name, 
                  b.id_shift as id_shift,
                  b.cnt as cnt, 
                  b.time as time,
                  shift.staff_id as staff_id
                  from (""" 
    q2 += q[:-1] + ") as b Inner join shift on b.id_shift = shift.id;"

    #print(execute_sql_script_and_return_result(q2))

    q3 = """select c.id as sale_id,
                  c.item_id as item_id, 
                  c.name as item_name, 
                  c.id_shift as id_shift,
                  c.cnt as cnt_items, 
                  c.time as sell_time,
                  c.staff_id as staff_id,
                  staff.name as staff_name,
                  staff.role as staff_role
                  from (""" 
    q3 += q2[:-1] + ") as c Inner join staff on c.staff_id = staff.id where c.staff_id in (" + str_cashiers_id + ");"

    
    res_sales = [c for c in self.__execute_sql_script_and_return_result(q3)];
    titles = ("sale_id","item_id", "item_name", "id_shift", "cnt_items", "sell_time", "staff_id", "staff_name", "staff_role")
    #tmp = tuple(get_column_name('sales'))
    res_sales.insert(0, titles)    
    return (True, res_sales)

  def __clear_table(self, table_name):
    cursor = self.__conn.cursor()
    q = "truncate table " + table_name + " cascade;"
    try:
      cursor.execute(q)
    finally:
      cursor.close()
      self.__conn.commit()

  def __init__(self):
    self.__is_start_session = False
    self.__current_id = -1
    self.__conn = psycopg2.connect(dbname="test6", user="root")
    self.__execute_sql_script("create table if not exists staff (id integer, login varchar, psw varchar, role varchar, name varchar, primary key (id), t_start date, t_end date);")
    self.__execute_sql_script("create table if not exists shift (id integer, t_start date, t_end date, staff_id integer, primary key (id), FOREIGN KEY (staff_id) REFERENCES staff (id))")
    self.__execute_sql_script("create table if not exists sales (id integer, item_id integer, id_shift integer, cnt integer, time date, FOREIGN KEY (id_shift) REFERENCES shift (id));")
    self.__execute_sql_script("create table if not exists types_of_coffee (id integer, name varchar, price integer, description varchar, primary key (id), t_start date, t_end date);")
    self.__execute_sql_script("create table if not exists sessions(id integer, login varchar, primary key (id), t_start date, t_end date);")
    self.clear_all_tables()
  def create_test_data(self):
    #создаем пользователей
    self.__add_new_person("Петя", "admin", "admin", "ADMIN")
    self.__add_new_person("Петя1", "admin1", "admin", "ADMIN")
    self.__add_new_person("Вася", "user", "user", "USER")
    self.__add_new_person("Коля", "user1", "user", "USER")
    self.__add_new_person("Коля1", "user2", "user", "USER")
    self.__add_new_person("Коля2", "user3", "user", "USER")
    self.__add_new_person("Коля3", "user4", "user", "USER")
    #создаем открытие и закрытия смен
    for i in range(1, 8):
      for j in range(1, 10):
        с = random.randint(1, 2)
        if (с == 1):
          self.__close_shift(i)
        else:
          self.__start_shift(i)
    #добавляем виды кофе (за уникальностью id и наполнением таблицы следит оператор/АИБ, от системы в задании этого не требовалось)
    self.__add_new_type_of_coffee("Латте", 150, "хороший  Латте")
    self.__add_new_type_of_coffee("Эспрессо", 100, "хороший  Эспрессо")
    self.__add_new_type_of_coffee("Капучино", 150, "хороший  Капучино")
    self.__add_new_type_of_coffee("Американо", 100, "хороший  Американо")
    #добавляем заказы
    self.__make_order(3, [[1, 3], [2, 5], [3, 4]])
    self.__make_order(7, [[1, 3], [2, 15], [3, 4]])
    self.__make_order(3, [[1, 3], [4, 55], [3]])
    self.__make_order(4, [[4, 3], [2, 5], [3]])
    return(True, "ок")

  def start_session(self, login, psw):
    if self.__is_start_session:
      return (False, "Закройте предыдущую сессию. После чего вы сможете запустить новую")
    else:
      q = "select id, psw from staff where login = '" + login + "';"
      #print(q)
      user = self.__execute_sql_script_and_return_result(q)
      #print(user)
      if len(user) == 0:
        return (False, "Данного пользователя нет в системе. Обратитесь к АИБ")
      if user[0][1] != psw: #считаем, что результат 1, так как таблица staff ведется вручную 
        return (False, "Неверный пароль")
      id = self.__get_next_id("sessions")
      self.__current_id = user[0][0]
      self.__is_start_session = True
      q = "insert into sessions (id, login, t_start, t_end) values (" + str(id) + ", '" + login +  "', CURRENT_TIMESTAMP, NULL);"
      #print(q)
      self.__execute_sql_script(q)
      return (True, "")

  
  def close_session(self):
    if not self.__is_start_session:
      return (False, " сессия не открыта")     
    else:
      tmp = self.__close_shift(self.__current_id)
      q = "update sessions set t_end = CURRENT_TIMESTAMP where id = " + str(self.__current_id) + ";"
      self.__execute_sql_script(q)
      self.__current_id = -1
      self.__is_start_session = False
      return(True, "")
  #def print_all_staff(self):
  #  self.__print_all_data_from_table("staff")
  #def print_all_type_of_coffee(self):
  #  self.__print_all_data_from_table("types_of_coffee")
  #def print_all_shift(self):
  #  self.__print_all_data_from_table("shift")
  #def print_all_sessions(self):
  #  self.__print_all_data_from_table("sessions")
  #def print_all_sales(self):
  #  self.__print_all_data_from_table("sales")
  def clear_all_tables(self):
    self.__clear_table("sales")
    self.__clear_table("shift")
    self.__clear_table("staff")
    self.__clear_table("types_of_coffee")
    self.__clear_table("sessions")
  def fix_connection(self):
    curs = self.__conn.cursor()
    curs.execute("ROLLBACK")
    self.__conn.commit()
  def start_shift(self):
    if not self.__is_start_session: 
      return (False, "Сессия не открыта. Перед тем, как открыть смену, запустите сесиию")    
    return self.__start_shift(self.__current_id)
  def close_shift(self):
    if not self.__is_start_session: 
      return (False, "Сессия не открыта. Перед тем, как закрыть смену, запустите сесиию")    
    return self.__close_shift(self.__current_id)
  def make_order(self, arr):
     if not self.__is_start_session: 
      return (False, "Сессия не открыта. Перед тем, как закрыть смену, запустите сесиию")    
     return self.__make_order(self.__current_id, arr)
  def get_sales(self, cashiers_id, shifts_id, items_id):
    if not self.__is_start_session: 
      return (False, "Сессия не открыта. Перед тем, как закрыть смену, запустите сесиию")
    return self.__get_sales(self.__current_id, cashiers_id, shifts_id, items_id)

Как работать с данным ноутбуком:
0. Создаем экземпляр системы учеты для киоска продажи кофе
1. Логинимся

  Тестово созданы следующие пользователи (login/psw):  
  - администраторы: admin/admin, admin1/admin; 
  - пользователи (кассиры): user/user, user1/user, user2/user, user3/user, user4/user.

Пример авторизации: ch.start_session("user", "user") 

3. Для каждого типа пользователей доступны свои функции. 

Для пользователей (кассиров):
  - запуск смены ( ch.start_shift() )
  - завершение смены ( ch.close_shift() )
  - оформить заказ ( ch.make_order([array]) )
    * функция make_order принимает на вход из заказов
    * заказ может быть: 
     * одноэлементным массивом с id заказанного кофе;
     * двухэлементным массивом с id заказанного кофе и его количеством;
    * таким образом ch.make_order([[1],[4,1],[3, 4]]) это заказ:
      * 1 кофе с id = 1
      * 1 кофе с id = 4
      * 4 чашки кофе с id = 3
  - функция просмотра всех заказов (get_sales) пользователю (кассиру) не доступна.
  
Для администраторов:
  - не доступны функции кассира (см выше)
  - доступна функция просмотра списка продаж с фильтрами

      ch.get_sales(staff_id, shifts_id, items_id), где
      * staff_id - массив id кассиров, по которым будет делаться выборка при формировании перечня заказов 
      * shifts_id - массив id смен кассиров, по которым будет делаться выборка при формировании перечня заказов 
      * shiftitems_id - массив id товаров (кофе), по которым будет делаться выборка при формировании перечня заказов

    Между условиями стоит конкатенация.

    Если вместо массива передавать пустой массив, то будет сделана выборка по всем возможным кассирам, сменам, товарам.

    Примеры:

      * ch.get_sales([6,4],[1,3],[4,2]) - список товаров с id 4 или 2, которые продали кассиры с id 6 или 4 во время смен 1 или 3
      * ch.get_sales([],[],[]) - список всех покупок

    В результате в первом элементе выводятся id столбцов, после чего сами записи о продажах.

  PS: таблицы с видами кофе (types_of_coffe) и с пользователями (staff) не ведутся автоматически (в задании этого не требовалось). Поэтому можно считать, что они ведутся организационным образом владельцем кафе/АИБами системы.

In [596]:
# создаем систему для киоска продажи кофе и заполняем ее тестовыми данными
ch = CoffeeHouse()
ch.create_test_data()

(True, 'ок')

In [597]:
#логинимся от кассира
ch.start_session("user", "user")

(True, '')

In [598]:
#проверяем, что второй подряд логин не работает
ch.start_session("user", "user")

(False, 'Закройте предыдущую сессию. После чего вы сможете запустить новую')

In [599]:
#запускаем смену 
ch.start_shift()

(False, 'Ваша смена уже запущена')

Она уже была запущена во время генерации тестовых данных

In [601]:
#делаем заказ
ch.make_order([[1],[2,1],[3]])

(True,
 ['insert into sales (id, item_id, id_shift, cnt, time) values (4, 1, 3, 0, CURRENT_TIMESTAMP);',
  'insert into sales (id, item_id, id_shift, cnt, time) values (4, 2, 3, 1, CURRENT_TIMESTAMP);',
  'insert into sales (id, item_id, id_shift, cnt, time) values (4, 3, 3, 0, CURRENT_TIMESTAMP);'])

In [602]:
ch.make_order([[4,20],[3,1],[3]])

(True,
 ['insert into sales (id, item_id, id_shift, cnt, time) values (5, 4, 3, 20, CURRENT_TIMESTAMP);',
  'insert into sales (id, item_id, id_shift, cnt, time) values (5, 3, 3, 1, CURRENT_TIMESTAMP);'])

In [604]:
#проверяем, что не можем посмотреть все продажи функцией админимтратора
ch.get_sales([],[],[1])

(False,
 'У вас нет прав на создлание заказа. Для обновления прав доступа обратитесь к АИБ системы')

In [605]:
#закрываем смену
ch.close_shift()

(True,
 'update shift set t_end = CURRENT_TIMESTAMP where staff_id = 3 and t_end is NULL;')

In [606]:
#закрываем сессию
ch.close_session()

(True, '')

In [607]:
#логинимся от администратора
ch.start_session("admin1", "admin")

(True, '')

In [608]:
#выводим заказы, оформленные кассирами 1 или 2 в любые смены для товаров 3 или 4
#ch.get_sales(staff_id, shifts_id, items_id)
ch.get_sales([3,1],[],[3,4])

(True,
 [('sale_id',
   'item_id',
   'item_name',
   'id_shift',
   'cnt_items',
   'sell_time',
   'staff_id',
   'staff_name',
   'staff_role'),
  (1, 3, 'Капучино', 3, 4, datetime.date(2021, 1, 19), 3, 'Вася', 'USER'),
  (5, 3, 'Капучино', 3, 1, datetime.date(2021, 1, 19), 3, 'Вася', 'USER'),
  (4, 3, 'Капучино', 3, 0, datetime.date(2021, 1, 19), 3, 'Вася', 'USER'),
  (2, 3, 'Капучино', 3, 0, datetime.date(2021, 1, 19), 3, 'Вася', 'USER'),
  (2, 4, 'Американо', 3, 55, datetime.date(2021, 1, 19), 3, 'Вася', 'USER'),
  (5, 4, 'Американо', 3, 20, datetime.date(2021, 1, 19), 3, 'Вася', 'USER')])

In [609]:
#ch.get_sales(staff_id, shifts_id, items_id)
#все продажи 
ch.get_sales([],[],[])

(True,
 [('sale_id',
   'item_id',
   'item_name',
   'id_shift',
   'cnt_items',
   'sell_time',
   'staff_id',
   'staff_name',
   'staff_role'),
  (1, 3, 'Капучино', 3, 4, datetime.date(2021, 1, 19), 3, 'Вася', 'USER'),
  (5, 3, 'Капучино', 3, 1, datetime.date(2021, 1, 19), 3, 'Вася', 'USER'),
  (2, 1, 'Латте', 3, 3, datetime.date(2021, 1, 19), 3, 'Вася', 'USER'),
  (4, 3, 'Капучино', 3, 0, datetime.date(2021, 1, 19), 3, 'Вася', 'USER'),
  (5, 4, 'Американо', 3, 20, datetime.date(2021, 1, 19), 3, 'Вася', 'USER'),
  (1, 2, 'Эспрессо', 3, 5, datetime.date(2021, 1, 19), 3, 'Вася', 'USER'),
  (4, 1, 'Латте', 3, 0, datetime.date(2021, 1, 19), 3, 'Вася', 'USER'),
  (3, 2, 'Эспрессо', 6, 5, datetime.date(2021, 1, 19), 4, 'Коля', 'USER'),
  (3, 3, 'Капучино', 6, 0, datetime.date(2021, 1, 19), 4, 'Коля', 'USER'),
  (1, 1, 'Латте', 3, 3, datetime.date(2021, 1, 19), 3, 'Вася', 'USER'),
  (2, 3, 'Капучино', 3, 0, datetime.date(2021, 1, 19), 3, 'Вася', 'USER'),
  (2, 4, 'Американо', 3, 55, datet